In [1]:
%run project_functions.ipynb

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [ ]:
from collections import deque

In [29]:
file_name = 'FB.txt'
csv_name = 'FB.csv'
T = 3
capital = 90000

In [3]:
#Features
company = stock_twits_text_parser(file_name)
company_metrics = stock_twits_metrics(company, T, file_name)

#Labels
company_return = return_over_period_T(csv_name,3)

#Number of Trees:
n = 400

#Cleaned Data
agg_data = feature_selector(company_metrics, company_return, 'close_return', 25, n)
company_data = split_data(agg_data, 0.7, 'all', 'close_return', 'bin')

#Split
X_train = company_data['X_train']
Y_train = company_data['Y_train']
X_test = company_data['X_test']
Y_test = company_data['Y_test']

In [4]:
#Fit Model
clf_rf = RandomForestClassifier().fit(X_train, Y_train)

#Predict
pred_test = clf_rf.predict(X_test)

In [59]:
def trader(predictions, period, metric, numerical_data, capital):
    metric.lower()
    target = metric + '_return'
    
    r = numerical_data[target]
    price = numerical_data[metric.upper()]
    
    action = deque()

    for i in range (len(pred_test)):
    
        cash = capital/3
    
        if(cash < price[i]):
            continue
    
        if(i < T):
            action.append(pred_test[i])
        else:
            #make new prediction for t+T
            action.append(pred_test[i])
        
            #calculate return based on prediction
            prediction = action.popleft()
        
            if(prediction == 1):
                #calculate long return
                profit = r[i-3]
            elif(prediction == -1):
                #calculate short return
                profit = -r[i-3]
            #else prediction will = 0, do nothing
            else:
                profit = 0
            #update capital value
            capital+=profit
            
            #DEGUG print(str(r[i-3]) + " " + str(profit))
        #DEBUG print(capital)
        
    return capital

In [60]:
trader(pred_test, T, 'close', company_return, 90000)

90000
90000
90000
89999.99842832149
89999.99884091562
90000.00988478333
90000.02331745188
90000.0527019148
90000.06478048861
90000.03893217516
90000.04934224537
90000.05189386141
90000.02814852801
90000.01237897426
90000.01354547475
90000.02119286705
90000.02973799602
90000.03249839516
90000.0511219422
90000.06054791147
90000.09642571409
90000.19214915836
90000.05344385696
90000.13692440209
90000.12379398115
90000.03072530808
89999.93421925989
89999.9085282638
89999.93638962472
89999.94869599737
89999.96300884541
89999.95120099302
89999.95520588793
89999.98118433317
90000.02333254206
89999.99518550723
90000.01586215428
90000.0058599911
89999.9906509313
90000.012642027
90000.02878842062
90000.05126039566
90000.06588830597
90000.07044340122
90000.05851248583
90000.06055639722
90000.11121088215
90000.02922798127
89999.99829130893
89999.97878679128
89999.91648147082
89999.870764008
89999.85042309473
89999.85400734584
89999.83762609951
89999.83288477424
89999.85301464393
89999.83517099754
8

89999.72328598138